# Titanic Model Deployment

The purpose of this notebook is to demonstrate how the model yielded by `titanic-ml.ipynb` can be deployed as a RESTful service on Kubernetes.

Required steps:

1. inject model into a Flask service:
    - create 'deploy' directory;
    - 'download' model locally;
    - 'download' Flask model wrapper;
    - use environment variable to pass model name to Flask service;
2. inject Flask service into Docker container:
    - build Docker image;
    - push image to registry;
3. deploy end-to-end service to Kubernetes using a Helm chart:
    - download the appropriate Helm chart;
    - set the appropriate parameter values for service naming, clusters, etc.; and,
    - deploy using Helm!

In [1]:
import os

import docker

## Deployment Configuration Parameters

In [24]:
MODEL_REPOSITORY = '../models'
SERVICE_NAME = 'titanic'
API_VERSION = '1'
DOCKER_IMAGE_REGISTRY = 'alexioannides'

## Copy Model to Flask Service Source Code Directory

Which is `py-sklearn-flask-ml-service` within the current directory.

In [26]:
latest_model = sorted(os.listdir(MODEL_REPOSITORY))[-1]
os.popen(f'cp {latest_model} deploy/py-sklearn-flask-ml-service/model.joblib')
print(f'{latest_model} --> deploy/py-sklearn-flask-ml-service/model.joblib')

titanic-ml-2019-02-11T17:48:28.joblib --> deploy/py-sklearn-flask-ml-service/model.joblib


## Write Service Configuration Parameters

We use a `.env` file in `py-sklearn-flask-ml-service` that will be copied to the Docker image, automatically loaded by Pipenv and used by the Flask service described in `py-sklearn-flask-ml-service` to define the REST API endpoint.

In [28]:
with open('py-sklearn-flask-ml-service/.env', 'w') as file:
    file.writelines(f'SERVICE_NAME={SERVICE_NAME}\n')
    file.writelines(f'API_VERSION={API_VERSION}\n')

Validate the contents of the `.env` file.

In [29]:
!cat py-sklearn-flask-ml-service/.env

SERVICE_NAME=titanic
API_VERSION=1


## Build Docker Image

Ensure that there is a Docker daemon up-and-runnning.

> Note, in production we will need a build server (e.g. Travis CI) to build images and push to registries.

In [35]:
image_tag = f'{DOCKER_IMAGE_REGISTRY}/{SERVICE_NAME}:latest'

docker_client = docker.from_env()
image, build_log = docker_client.images.build(
    path='py-sklearn-flask-ml-service', tag=image_tag, rm=True)

docker_client.images.prune()

for line in build_log:
    print(line)

{'stream': 'Step 1/7 : FROM python:3.7-slim'}
{'stream': '\n'}
{'stream': ' ---> 12c44ed85032\n'}
{'stream': 'Step 2/7 : WORKDIR /usr/src/app'}
{'stream': '\n'}
{'stream': ' ---> Using cache\n'}
{'stream': ' ---> 987983ee8d98\n'}
{'stream': 'Step 3/7 : COPY . .'}
{'stream': '\n'}
{'stream': ' ---> Using cache\n'}
{'stream': ' ---> 0ed602e98f4e\n'}
{'stream': 'Step 4/7 : EXPOSE 5000'}
{'stream': '\n'}
{'stream': ' ---> Using cache\n'}
{'stream': ' ---> c86b62a88f5c\n'}
{'stream': 'Step 5/7 : RUN pip install pipenv'}
{'stream': '\n'}
{'stream': ' ---> Using cache\n'}
{'stream': ' ---> 48a84b9efda6\n'}
{'stream': 'Step 6/7 : RUN pipenv install'}
{'stream': '\n'}
{'stream': ' ---> Using cache\n'}
{'stream': ' ---> 5aa23904911f\n'}
{'stream': 'Step 7/7 : ENTRYPOINT ["pipenv", "run", "python", "api.py"]'}
{'stream': '\n'}
{'stream': ' ---> Using cache\n'}
{'stream': ' ---> 7ecf88006784\n'}
{'aux': {'ID': 'sha256:7ecf88006784076c94f7e1e69f5739cef092c7f83b23b9117145550533dfddb7'}}
{'stream': '

## Push Image to DockerHub

- Will need to use `docker_client.login()` on a build server.

In [34]:
push_log = docker_client.images.push(image_tag)
print(push_log)

{"status":"The push refers to repository [docker.io/alexioannides/titanic]"}
{"status":"Preparing","progressDetail":{},"id":"2b117c418425"}
{"status":"Preparing","progressDetail":{},"id":"a78c9ad3c595"}
{"status":"Preparing","progressDetail":{},"id":"9449b123dded"}
{"status":"Preparing","progressDetail":{},"id":"f49f129dd410"}
{"status":"Preparing","progressDetail":{},"id":"be1cb44a8fa5"}
{"status":"Preparing","progressDetail":{},"id":"94a02319f331"}
{"status":"Preparing","progressDetail":{},"id":"c22eb8781541"}
{"status":"Preparing","progressDetail":{},"id":"622d7e308e90"}
{"status":"Preparing","progressDetail":{},"id":"0a07e81f5da3"}
{"status":"Waiting","progressDetail":{},"id":"94a02319f331"}
{"status":"Waiting","progressDetail":{},"id":"c22eb8781541"}
{"status":"Waiting","progressDetail":{},"id":"622d7e308e90"}
{"status":"Waiting","progressDetail":{},"id":"0a07e81f5da3"}
{"status":"Layer already exists","progressDetail":{},"id":"2b117c418425"}
{"status":"Layer already exists","prog

## Deploy to Kubernetes using Helm Charts

We use the Helm chart located at `k8s-helm-ml-prediction-app` and discussed [here](https://github.com/AlexIoannides/kubernetes-ml-ops) to deploy our ML REST API as a fully-managed, self-healing and load balanced service on Kubernetes.

Note, ensure that `kubectl config current-context` is set the Kubernetes cluster we wish to deploy to and that this cluster has an operational Helm Tiller.

In [38]:
!helm install k8s-helm-ml-prediction-app \
    --set app.name="$SERVICE_NAME-survival-prediction" \
    --set app.namespace="$SERVICE_NAME" \
    --set app.image="$image_tag"

NAME:   newbie-stingray
LAST DEPLOYED: Wed Feb 13 01:21:24 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Pod(related)
NAME                                  READY  STATUS             RESTARTS  AGE
titanic-survival-prediction-rc-4tqd2  0/1    ContainerCreating  0         0s
titanic-survival-prediction-rc-t9sv9  0/1    ContainerCreating  0         0s

==> v1/Namespace
NAME     STATUS  AGE
titanic  Active  1s

==> v1/Service
NAME                            TYPE          CLUSTER-IP     EXTERNAL-IP  PORT(S)         AGE
titanic-survival-prediction-lb  LoadBalancer  10.108.72.116  <pending>    5000:31187/TCP  0s

==> v1/ReplicationController
NAME                            DESIRED  CURRENT  READY  AGE
titanic-survival-prediction-rc  2        2        0      0s


NOTES:
Thank you for installing helm-ml-score-app.

Your release is named newbie-stingray.

To learn more about the release, try:

  $ helm status newbie-stingray
  $ helm get newbie-stingray



## Test Prediction Service

We will assume that Minikube is being used for local testing, in which case we will need to ask it for the URL of its 'virtual load balancer' used for our service.

In [98]:
!minikube service list

|-------------|--------------------------------|-----------------------------|
|  NAMESPACE  |              NAME              |             URL             |
|-------------|--------------------------------|-----------------------------|
| default     | kubernetes                     | No node port                |
| kube-system | kube-dns                       | No node port                |
| kube-system | kubernetes-dashboard           | No node port                |
| kube-system | tiller-deploy                  | No node port                |
| titanic     | titanic-survival-prediction-lb | http://192.168.99.114:30438 |
|-------------|--------------------------------|-----------------------------|


We can then take the appropriate URL from the above and test our titanic prediction service!

In [100]:
!curl http://192.168.99.114:30438/titanic/v1/predict \
        --request POST\
        --header 'Content-Type: application/json' \
        --data '{"Pclass": [1], "Sex": ["male"], "Age": [32], "SibSp": [1],                      "Parch": [0], "Fare": [100], "Embarked": ["S"]}'

{"prediction":[1]}
